In [ ]:
import random
import folium

import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)

from sklearn.cluster import DBSCAN

import statsmodels.api as sm
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [33]:
df = pd.read_csv("C:/Users/Настя/YandexDisk-n4skolesnikova/HSE 4th year/Graduation Thesis/ACCIDENT_LEVEL_DATA.csv")
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
df.head()

(440132, 81)


,date,COORD_L,COORD_W,dor,dor_k,s_dtp,RTA_vehicle_number,RTA_participants_number,RTA_number,number_of_deaths,...,accident_month,accident_weekday,accident_season,is_weekend,RTA_hour,is_night,is_peak_hour,severity,severity_detailed,is_toll
0,31.01.2015,81.151944,53.740000,Романово - Завьялово - Баево - Камень-на-Оби,5.0,610,1,3,161242174,0,...,1,5,1,1,9,0,1,medium,injury_with_hospitalization,0
1,30.01.2015,85.018056,51.684444,Куяган - Куяча - Тоурак,6.0,200,2,3,161105683,0,...,1,4,1,0,14,0,0,medium,injury_with_hospitalization,0
2,30.01.2015,81.250000,53.818056,Барнаул - Камень-на-Оби - граница Новосибирско...,5.0,210,2,3,161763431,0,...,1,4,1,0,17,0,1,light,injury_no_hospitalization,0
3,24.01.2015,51.000000,84.000000,Быканов Мост - Солоновка - Солонешное - границ...,7.0,610,1,2,160331994,0,...,1,5,1,1,19,0,0,light,injury_no_hospitalization,0
4,23.01.2015,84.000000,53.000000,Быканов Мост - Солоновка - Солонешное - границ...,7.0,600,1,2,160213415,1,...,1,4,1,0,21,0,0,severe,death_at_scene,0


In [34]:
replace_map = {
    'light': '1',
    'medium': '2',
    'severe': '3'
}

df['severity_encode'] = df['severity'].replace(replace_map)
df['severity_encode'] = df['severity_encode'].astype(int)
df['severity_encode'].unique()

array([2, 1, 3])

In [35]:
df = df.drop(columns=['severity_detailed', 'severity', 'num_fatal_traffic_violations'])

In [ ]:
toll_roads_df = df[df['is_toll'] == 1]
free_roads_df = df[df['is_toll'] == 0]

dbscan_toll = DBSCAN(eps=0.2/6371.0, min_samples=5, metric="haversine")
toll_roads_df["cluster"] = dbscan_toll.fit_predict(np.radians(toll_roads_df[["COORD_W", "COORD_L"]].values))

dbscan_free = DBSCAN(eps=0.3/6371.0, min_samples=5, metric="haversine")  # larger eps for free ones as more observations
free_roads_df["cluster"] = dbscan_free.fit_predict(np.radians(free_roads_df[["COORD_W", "COORD_L"]].values))

df = pd.concat([toll_roads_df, free_roads_df], ignore_index=True)

C:\Users\Настя\AppData\Local\Temp\ipykernel_24944\1707238632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toll_roads_df["cluster"] = dbscan_toll.fit_predict(np.radians(toll_roads_df[["COORD_W", "COORD_L"]].values))
C:\Users\Настя\AppData\Local\Temp\ipykernel_24944\1707238632.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  free_roads_df["cluster"] = dbscan_free.fit_predict(np.radians(free_roads_df[["COORD_W", "COORD_L"]].values))


In [ ]:
# Negative examples (clusters with label -1)
negative_examples = df[df["cluster"] == -1]

# Positive examples (clusters with labels 0, 1, 2, ...)
positive_examples = df[df["cluster"] != -1]

# Hypotheses testing

In [39]:
imsbalance = positive_examples['is_toll'].sum() / positive_examples.shape[0]
print(f"Class imbalance in hotspot dataset: {imsbalance}")

Class imbalance in hotspot dataset: 0.08205811707518727


In [ ]:
sample_weights = compute_sample_weight(class_weight='balanced', y=positive_examples['is_toll'])
df_weighted = positive_examples.loc[positive_examples.index.repeat(sample_weights.round().astype(int))]

X = df_weighted[[
    'is_toll', 'RTA_vehicle_number', 'RTA_participants_number',
    'num_drunk_participants', 'guilty_drivers_avg_experience',
    'drivers_avg_experience', 'share_guilty_vehicles',
    'lighting_cat', 'road_object_type_cat', 'speeding', 'signal_violation',
    'pedestrian_related', 'wrong_way_driving', 'num_front_wheel_drive_vehicles',
    'avg_vehicles_age', 'dor_z_cat', 'DTPV_cat', 'OBJ_DTP_cat', 'factor_cat'
]]

X = sm.add_constant(X)

y = df_weighted['severity_encode']

model = sm.MNLogit(y, X)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.945331
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:        severity_encode   No. Observations:               197868
Model:                        MNLogit   Df Residuals:                   197828
Method:                           MLE   Df Model:                           38
Date:                Sun, 20 Apr 2025   Pseudo R-squ.:                 0.05397
Time:                        20:42:47   Log-Likelihood:            -1.8705e+05
converged:                       True   LL-Null:                   -1.9772e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
             severity_encode=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             39.4074      1.573     25.047     

Мы снова подтвердили гипотезу о том, что вероятность тяжёлой аварии на платных дорогах выше, чем на бесплатных.

Также, сравнив коэффициенты для признака `is_toll` между моделью для всего датасета (см. файл `severity_hypotheses.ipynb`) и моделью для горячих точек выше, можно отметить, что для горячих точек коэффициент для `is_toll` значительно выше: для средней тяжести аварий (severity_encode = 2) — 0.3235, а для тяжёлых аварий (severity_encode = 3) — 0.4115. В то время как для всех наблюдений коэффициенты значительно ниже: для средней тяжести — 0.2609, а для тяжёлых аварий — 0.2564. Это указывает на то, что влияние признака `is_toll` на тяжесть аварий в горячих точках усиливается по сравнению с общими дорогами.

Это подводит нас к созданию модели, которая сможет предсказывать горячие точки по координатам и дополнительным признакам.

# Hot spot predictor model

The model works better with info about driver behavior on the roads and other accidents variables, but we must proceed from the fact that when opening a new section of road we have no information about accidents on it, so we will select features that reflect the geographical location and characteristics of the road section itself.

In [62]:
features = ["COORD_L", "COORD_W", "road_object_type_cat", "dor_z_cat", "OBJ_DTP_cat", "CHOM_cat", "ndu_cat", "factor_cat", "dor_k", "is_toll"]

X_model = df[features]

# Target variable: hotspots (1 = hotspot, 0 = not hotspot)
y_model = np.where(df["cluster"] != -1, 1, 0)

X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 (macro):", f1_score(y_test, y_pred, average='macro'))

Accuracy: 0.8456382700762266
F1 (macro): 0.820213298890597


Gradient boosting was also tried, but it resulted in 5% less quality on both metrics.

Example of prediction:

In [63]:
new_data = pd.DataFrame({
    "COORD_L": [55.7558],
    "COORD_W": [37.6173],
    "road_object_type_cat": [2],
    "dor_z_cat": [1],
    "OBJ_DTP_cat": [3],
    "CHOM_cat": [0],
    "ndu_cat": [1],
    "factor_cat": [0],
    "dor_k": [1],
    "is_toll": [0]
})

new_prediction = rf_model.predict(new_data)

if new_prediction[0] == 1:
    print("This is a hot spot (high-risk area).")
else:
    print("This is not a hot spot (low-risk area).")

This is not a hot spot (low-risk area).


In [64]:
m = folium.Map(location=[55.7558, 37.6173], zoom_start=10)

# Если точка горячая, красный маркер, если нет — синий
color = "red" if new_prediction[0] == 1 else "blue"

folium.CircleMarker(
    location=[new_data["COORD_L"][0], new_data["COORD_W"][0]],
    radius=6,
    color=color,
    fill=True,
    fill_color=color,
    fill_opacity=0.6
).add_to(m)

m

Since the model is trained on both toll and non-toll roads, and includes the `'is_toll'` feature along with only those features related to the geographical location and road characteristics, it can be useful for predicting dangerous areas on new toll roads. Observations from non-toll roads, unlike toll roads, almost entirely cover the country's territory, which will allow the model to identify hotspots on a new toll road, even if it is geographically far from previously built toll highways.

Let's check it! We will try to train the model on data excluding the toll road M-1 and see how well the model performs in identifying hotspots for this road.

In [ ]:
df_train = df[~df['dor'].str.contains('М-1', case=False, na=False)]
X_train = df_train[features]
y_train = np.where(df_train["cluster"] != -1, 1, 0)  # 1 = hotspot, 0 = not hotspot

df_m1 = df[df['dor'].str.contains('М-1', case=False, na=False)]
X_test = df_m1[features]
y_test = np.where(df_m1["cluster"] != -1, 1, 0)  # 1 = hotspot, 0 = not hotspot

In [44]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_m1 = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_m1)
f1_macro = f1_score(y_test, y_pred_m1, average='macro')

print("Accuracy for М-1:", accuracy)
print("F1 (macro) for М-1:", f1_macro)

Accuracy for М-1: 0.68481455563331
F1 (macro) for М-1: 0.6166441473508499
